In [3]:
from z import *
from trainer import *
from torchvision import datasets, transforms
from torchsummary import summary

In [4]:
train_dataset = datasets.MNIST('./MNIST', train=True, download=True,
                               transform=transforms.Compose([transforms.ToTensor()]))
test_dataset = datasets.MNIST('./MNIST', train=False, 
                              transform=transforms.Compose([transforms.ToTensor()]))

Extracting ./MNIST\MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST\MNIST\raw


Extracting ./MNIST\MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST\MNIST\raw


Extracting ./MNIST\MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST\MNIST\raw


Extracting ./MNIST\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST\MNIST\raw
Processing...


C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [5]:
model = nn.Sequential(
    ZConv2d(1, 16, 4, 5),
    SoftPool2d(16, 2, 2),
    ZConv2d(16, 32, 4, 5),
    SoftPool2d(32, 2, 2),
    nn.Flatten(),
    nn.Dropout(0.5),
    ZLinear(32 * 4 * 4, 10, 4)
).to(0)

In [6]:
summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 24, 24]           1,664
           ZConv2d-2           [-1, 16, 24, 24]               0
        SoftPool2d-3           [-1, 16, 12, 12]               0
            Conv2d-4            [-1, 128, 8, 8]          51,328
           ZConv2d-5             [-1, 32, 8, 8]               0
        SoftPool2d-6             [-1, 32, 4, 4]               0
           Flatten-7                  [-1, 512]               0
           Dropout-8                  [-1, 512]               0
            Linear-9                   [-1, 40]          20,520
          ZLinear-10                   [-1, 10]               0
Total params: 73,512
Trainable params: 73,512
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.46
Params size (MB): 0.28
Estimated Tot

In [7]:
def loss_func(model, batch, scope):
    x, y = batch
    _y = model(x)
    loss = F.cross_entropy(_y, y)
    scope["metrics"]["Accuracy"] = float(torch.sum((torch.argmax(_y, dim=1) == y).type(torch.float))) / len(scope["dataset"])
    return loss, _y

In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [9]:
acc = 0
def on_epoch(scope):
    global acc
    _acc = scope["val_metrics"]["Accuracy"]
    if _acc > acc:
        acc = _acc
        torch.save(model.state_dict(), "mnist.model")
        print("Model saved!")

In [10]:
train(model, loss_func, train_dataset, test_dataset, optimizer, device=0, epochs=100, batch_size=256, on_val_epoch=on_epoch)

  1%|█▎                                                                                                                                                   | 2/235 [00:00<00:17, 13.26it/s]

Epoch #1


  8%|███████████▎                                                                                                                                          | 3/40 [00:00<00:01, 29.94it/s]

	Train Loss = 0.519930924967689
	Train Accuracy = 0.8426833333333346


  0%|                                                                                                                                                             | 0/235 [00:00<?, ?it/s]

...

	Train Loss = 0.007546871069002841
	Train Accuracy = 0.9976333333333292


  1%|█▎                                                                                                                                                   | 2/235 [00:00<00:15, 15.21it/s]

	Validation Loss = 0.02007298491143152
	Validation Accuracy = 0.9939999999999994
Epoch #83


 26%|█████████████████████████████████████▊                                                                                                              | 60/235 [00:03<00:10, 16.22it/s]


KeyboardInterrupt: 

In [11]:
print("Best Accuracy = " + str(acc))

Best Accuracy = 0.9957999999999994
